LSTM Model with incremental learning to improve 15mins timeframe predictions of crypto assets

In [1]:
!pip install ccxt pandas numpy scikit-learn tensorflow pandas-ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.9/117.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 16.5 MB/s eta 0:00:00
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=13f33d4aa4fa79bfdca22fd10a0656ce039c3862c9249282a41babe629d9983e
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas-ta
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.11
    Uninstalling aiohttp-3.11.11:
      Successfully uninstalled aiohttp-3.11.11


In [2]:
import os
import numpy as np
import pandas as pd
import ccxt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from datetime import datetime
import joblib

# File paths for model and scalers
MODEL_FILE = "lstm_model.h5"
SCALER_FILE = "scaler.pkl"
TARGET_SCALER_FILE = "target_scaler.pkl"

# Fetch historical data
def fetch_historical_data(exchange, symbol, limit=200):
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe='15m', limit=limit)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        return df
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

# Preprocess data
def preprocess_data(df, sequence_length=50):
    # Feature scaling
    scaler = MinMaxScaler()
    target_scaler = MinMaxScaler()

    features = df[['open', 'high', 'low', 'close', 'volume']]
    scaled_features = scaler.fit_transform(features)

    # Normalize target column (close price)
    scaled_target = target_scaler.fit_transform(df[['close']])

    # Save scalers for future use
    joblib.dump(scaler, SCALER_FILE)
    joblib.dump(target_scaler, TARGET_SCALER_FILE)

    # Create sequences
    X, y = [], []
    for i in range(len(scaled_features) - sequence_length):
        X.append(scaled_features[i:i + sequence_length])
        y.append(scaled_target[i + sequence_length])
    return np.array(X), np.array(y), scaler, target_scaler

# Load or create LSTM model
# Load or create LSTM model
def create_or_load_model(input_shape):
    if os.path.exists(MODEL_FILE):
        try:
            print("Loading existing LSTM model...")
            model = load_model(MODEL_FILE, custom_objects={"mse": "mse"})
            # Check for shape mismatch
            if model.input_shape[1:] != input_shape:
                print("Model input shape mismatch. Recreating the model...")
                model = None
        except Exception as e:
            print(f"Error loading model: {e}. Recreating the model...")
            model = None
    else:
        model = None

    if model is None:
        print("Creating a new LSTM model...")
        model = Sequential([
            LSTM(50, return_sequences=True, input_shape=input_shape),
            LSTM(50),
            Dense(1)
        ])

    # Reset optimizer and recompile
    print("Recompiling the model with a new optimizer...")
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model


# Train the model
def train_model(model, X, y, epochs=10, batch_size=32):
    model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=1)
    model.save(MODEL_FILE)
    return model

# Predict with LSTM
def predict(model, X, target_scaler):
    predictions = model.predict(X)
    return target_scaler.inverse_transform(predictions)

# Generate trading signals
def generate_signals(exchange, assets):
    sequence_length = 50
    results = []

    for asset in assets:
        print(f"Processing {asset}...")
        df = fetch_historical_data(exchange, asset)
        if df is None or len(df) < sequence_length:
            print(f"Insufficient data for {asset}. Skipping.")
            continue

        X, y, scaler, target_scaler = preprocess_data(df, sequence_length)
        model = create_or_load_model((sequence_length, X.shape[2]))
        model = train_model(model, X, y)

        # Use the latest sequence for prediction
        latest_sequence = X[-1].reshape(1, *X.shape[1:])
        predicted_price = predict(model, latest_sequence, target_scaler)

        current_price = df['close'].iloc[-1]
        signal = "Buy" if predicted_price > current_price else "Sell"

        results.append({
            "asset": asset,
            "current_price": current_price,
            "predicted_price": predicted_price[0][0],
            "signal": signal
        })

    return results

# Main script
exchange = ccxt.kucoin()
assets = ["BTC/USDT", "ETH/USDT", "ADA/USDT", "XRP/USDT", "LINK/USDT", "COOKIE/USDT",
          "SOL/USDT" , "AVAX/USDT" , "CRV/USDT" , "CFX/USDT" , "INJ/USDT" , "DOGE/USDT" ,
          "KAS/USDT" , "NEAR/USDT" , "FET/USDT"]

signals = generate_signals(exchange, assets)

# Display results
print("\n--- Trading Signals ---")
for signal in signals:
    print(f"Asset: {signal['asset']}")
    print(f"  Current Price: ${signal['current_price']:.2f}")
    print(f"  Predicted Price: ${signal['predicted_price']:.2f}")
    print(f"  Signal: {signal['signal']}")


Processing BTC/USDT...
Creating a new LSTM model...
Recompiling the model with a new optimizer...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step - loss: 0.4809
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0345
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0594
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0198
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.0313
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - loss: 0.0141
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - loss: 0.0180
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0136
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 0.0136
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0147


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step
Processing ETH/USDT...


Loading existing LSTM model...
Recompiling the model with a new optimizer...
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - loss: 0.0240
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0252
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0168
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0143
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0141
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0157
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0101
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0123
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0109
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0088


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step
Processing NEAR/USDT...


Loading existing LSTM model...
Recompiling the model with a new optimizer...
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - loss: 0.0045
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0030
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0032
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0028
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0031
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0032
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0025
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0029
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0025
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0023


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 462ms/step
Processing FET/USDT...


Loading existing LSTM model...
Recompiling the model with a new optimizer...
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - loss: 0.0021
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0018
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0018
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0015
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0015
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0013
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0013
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0011
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0013
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0012


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step

--- Trading Signals ---
Asset: BTC/USDT
  Current Price: $102206.70
  Predicted Price: $102054.05
  Signal: Sell
Asset: ETH/USDT
  Current Price: $3121.04
  Predicted Price: $3128.38
  Signal: Buy
Asset: ADA/USDT
  Current Price: $0.94
  Predicted Price: $0.94
  Signal: Buy
Asset: XRP/USDT
  Current Price: $3.11
  Predicted Price: $3.11
  Signal: Sell
Asset: LINK/USDT
  Current Price: $23.18
  Predicted Price: $23.24
  Signal: Buy
Asset: COOKIE/USDT
  Current Price: $0.26
  Predicted Price: $0.26
  Signal: Sell
Asset: SOL/USDT
  Current Price: $230.61
  Predicted Price: $231.12
  Signal: Buy
Asset: AVAX/USDT
  Current Price: $33.01
  Predicted Price: $33.07
  Signal: Buy
Asset: CRV/USDT
  Current Price: $0.70
  Predicted Price: $0.69
  Signal: Sell
Asset: CFX/USDT
  Current Price: $0.15
  Predicted Price: $0.15
  Signal: Buy
Asset: INJ/USDT
  Current Price: $18.36
  Predicted Price: $18.34
  Signal: Sell
Asset: DOGE/USDT
  Current Price: $0.33
  

# 15M and 4H Timeframes

In [3]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import ccxt
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

# Initialize KuCoin exchange using CCXT
exchange = ccxt.kucoin()

# Fetch historical data from KuCoin
def fetch_historical_data(symbol, timeframe, limit=500):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
    return df

# Add technical indicators
def add_technical_indicators(df):
    df['RSI'] = df.ta.rsi(close='close', length=14)
    df['ATR'] = df.ta.atr(high='high', low='low', close='close', length=14)
    df['OBV'] = df.ta.obv(close='close', volume='volume')
    df['SMA_50'] = df.ta.sma(close='close', length=50)
    df['EMA_50'] = df.ta.ema(close='close', length=50)
    return df

# Preprocess data
def preprocess_data(df, sequence_length=50):
    scaler = MinMaxScaler()
    features = df[['close', 'RSI', 'ATR', 'OBV', 'SMA_50', 'EMA_50']].dropna()
    scaled_features = scaler.fit_transform(features)
    X, y = [], []
    for i in range(len(scaled_features) - sequence_length):
        X.append(scaled_features[i:i + sequence_length])
        y.append(scaled_features[i + sequence_length, 0])  # Predicting the 'close' price
    return np.array(X), np.array(y), scaler

# Build LSTM model
def build_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        LSTM(50),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Predict price
def predict_price(symbol, sequence_length=50):
    df_15m = fetch_historical_data(symbol, '15m', limit=500)
    df_4h = fetch_historical_data(symbol, '4h', limit=500)

    df_15m = add_technical_indicators(df_15m)
    df_4h = add_technical_indicators(df_4h)

    X_15m, _, scaler_15m = preprocess_data(df_15m, sequence_length)
    X_4h, _, scaler_4h = preprocess_data(df_4h, sequence_length)

    model_15m = build_model((X_15m.shape[1], X_15m.shape[2]))
    model_4h = build_model((X_4h.shape[1], X_4h.shape[2]))

    model_15m.fit(X_15m, X_15m[:, -1, 0], epochs=5, batch_size=32, verbose=1)
    model_4h.fit(X_4h, X_4h[:, -1, 0], epochs=5, batch_size=32, verbose=1)

    latest_sequence_15m = X_15m[-1].reshape(1, X_15m.shape[1], X_15m.shape[2])
    latest_sequence_4h = X_4h[-1].reshape(1, X_4h.shape[1], X_4h.shape[2])

    predicted_price_15m = scaler_15m.inverse_transform([[model_15m.predict(latest_sequence_15m)[0][0], 0, 0, 0, 0, 0]])[0][0]
    predicted_price_4h = scaler_4h.inverse_transform([[model_4h.predict(latest_sequence_4h)[0][0], 0, 0, 0, 0, 0]])[0][0]

    rsi = df_15m['RSI'].iloc[-1]
    atr = df_15m['ATR'].iloc[-1]
    obv = df_15m['OBV'].iloc[-1]

    return {
        'symbol': symbol,
        'current_price': df_15m['close'].iloc[-1],
        '15m_prediction': predicted_price_15m,
        '4h_prediction': predicted_price_4h,
        'RSI': rsi,
        'ATR': atr,
        'OBV': obv
    }

# Main function
symbols = ['BTC/USDT', 'ETH/USDT', 'ADA/USDT' , 'FET/USDT' , 'NEAR/USDT' , 'AVAX/USDT' , 'DOGE/USDT' ,
           'XRP/USDT' , 'LINK/USDT' , 'INJ/USDT' , 'COOKIE/USDT' , 'KAS/USDT', 'SOL/USDT' , 'LTC/USDT' ,
           'MNT/USDT' ,'AAVE/USDT' , 'JUP/USDT']  # Replace with desired trading pairs
for symbol in symbols:
    result = predict_price(symbol)
    print(f"Symbol: {result['symbol']}")
    print(f"Current Price: {result['current_price']:.2f}")
    print(f"15m Prediction: {result['15m_prediction']:.2f}")
    print(f"4h Prediction: {result['4h_prediction']:.2f}")
    print(f"RSI: {result['RSI']:.2f}")
    print(f"ATR: {result['ATR']:.2f}")
    print(f"OBV: {result['OBV']:.2f}")
    print("\n")


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 77ms/step - loss: 0.0742
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - loss: 0.0137
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.0045
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0031
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.0027
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.1298
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0186
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0140
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0108
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 0.0076
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 462ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 482ms/step
Symbol: BTC/USDT
Current Price: 102208.00
15m Prediction: 102586.67
4h Prediction: 100777.62
RSI: 59.75
ATR: 263.66
OBV: -1510.45


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - loss: 0.1390
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0100
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0058
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0034
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0022
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 70ms/step - loss: 0.1233
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - loss: 0.0289
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - loss: 0.0198
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0165
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0115
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 339ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step
Symbol: ETH/USDT
Current Price: 3120.18
15m Prediction: 3152.84
4h Prediction: 3177.72
RSI: 51.17
ATR: 12.35
OBV: -22152.73


Epoch 1/5


...

13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - loss: 0.1186
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - loss: 0.0170
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - loss: 0.0075
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - loss: 0.0043
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.0036
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step - loss: 0.2728
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0154
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0099
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0070
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0065
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step
Symbol: MNT/USDT
Current Price: 1.09
15m Prediction: 1.10
4h Prediction: 1.14
RSI: 54.95
ATR: 0.00
OBV: 65397.33


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - loss: 0.1016
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0130
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0052
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0038
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0033
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.2626
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0223
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0106
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0080
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - loss: 0.0062
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 477ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 362ms/step
Symbol: AAVE/USDT
Current Price: 296.01
15m Prediction: 296.71
4h Prediction: 312.21
RSI: 68.33
ATR: 2.09
OBV: -6252.56


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.1688
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0134
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0104
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0066
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0069
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 70ms/step - loss: 0.0560
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0186
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0090
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0074
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0075
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step
Symbol: JUP/USDT
Current Price: 1.10
15m Prediction: 1.13
4h Prediction: 1.09
RSI: 32.53
ATR: 0.01
OBV: 5163456.07




# 1D and 1Week Timeframes

In [4]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import ccxt
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# Initialize KuCoin exchange using CCXT
exchange = ccxt.kucoin()

# Fetch historical data from KuCoin
def fetch_historical_data(symbol, timeframe, limit=500):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
    return df

# Add technical indicators
def add_technical_indicators(df):
    df['RSI'] = ta.rsi(df['close'], length=14)  # RSI
    df['ATR'] = ta.atr(df['high'], df['low'], df['close'], length=14)  # ATR
    df['OBV'] = ta.obv(df['close'], df['volume'])  # OBV
    df['SMA_50'] = ta.sma(df['close'], length=50)  # Simple Moving Average (50)
    df['EMA_50'] = ta.ema(df['close'], length=50)  # Exponential Moving Average (50)
    df = df.dropna()
    return df


# Preprocess data
def preprocess_data(df, sequence_length=50):
    scaler = MinMaxScaler()
    features = df[['close', 'RSI', 'ATR', 'OBV', 'SMA_50', 'EMA_50']].dropna()
    scaled_features = scaler.fit_transform(features)
    X, y = [], []
    for i in range(len(scaled_features) - sequence_length):
        X.append(scaled_features[i:i + sequence_length])
        y.append(scaled_features[i + sequence_length, 0])  # Predicting the 'close' price
    return np.array(X), np.array(y), scaler

# Build LSTM model
def build_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        LSTM(50),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Predict price
def predict_price(symbol, sequence_length=50):
    df_1d = fetch_historical_data(symbol, '1d', limit=500)
    df_1w = fetch_historical_data(symbol, '1w', limit=500)

    df_1d = add_technical_indicators(df_1d)
    df_1w = add_technical_indicators(df_1w)

    X_1d, _, scaler_1d = preprocess_data(df_1d, sequence_length)
    X_1w, _, scaler_1w = preprocess_data(df_1w, sequence_length)

    model_1d = build_model((X_1d.shape[1], X_1d.shape[2]))
    model_1w = build_model((X_1w.shape[1], X_1w.shape[2]))

    model_1d.fit(X_1d, X_1d[:, -1, 0], epochs=5, batch_size=32, verbose=1)
    model_1w.fit(X_1w, X_1w[:, -1, 0], epochs=5, batch_size=32, verbose=1)

    latest_sequence_1d = X_1d[-1].reshape(1, X_1d.shape[1], X_1d.shape[2])
    latest_sequence_1w = X_1w[-1].reshape(1, X_1w.shape[1], X_1w.shape[2])

    predicted_price_1d = scaler_1d.inverse_transform([[model_1d.predict(latest_sequence_1d)[0][0], 0, 0, 0, 0, 0]])[0][0]
    predicted_price_1w = scaler_1w.inverse_transform([[model_1w.predict(latest_sequence_1w)[0][0], 0, 0, 0, 0, 0]])[0][0]

    rsi = df_1d['RSI'].iloc[-1]
    atr = df_1d['ATR'].iloc[-1]
    obv = df_1d['OBV'].iloc[-1]

    return {
        'symbol': symbol,
        'current_price': df_1d['close'].iloc[-1],
        '1d_prediction': predicted_price_1d,
        '1w_prediction': predicted_price_1w,
        'RSI': rsi,
        'ATR': atr,
        'OBV': obv
    }

# Main function
symbols = ['BTC/USDT', 'ETH/USDT', 'ADA/USDT' , 'FET/USDT' , 'NEAR/USDT' , 'AVAX/USDT' , 'DOGE/USDT' ,
           'XRP/USDT' , 'LINK/USDT' , 'INJ/USDT']  # Replace with desired trading pairs
for symbol in symbols:
    result = predict_price(symbol)
    print(f"Symbol: {result['symbol']}")
    print(f"Current Price: {result['current_price']:.2f}")
    print(f"1D Prediction: {result['1d_prediction']:.2f}")
    print(f"1W Prediction: {result['1w_prediction']:.2f}")
    print(f"RSI: {result['RSI']:.2f}")
    print(f"ATR: {result['ATR']:.2f}")
    print(f"OBV: {result['OBV']:.2f}")
    print("\n")


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - loss: 0.1684
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0226
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0116
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0071
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0046
Epoch 1/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.0521
Epoch 2/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0162
Epoch 3/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0083
Epoch 4/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0073
Epoch 5/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0050
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 478ms/step
Symbol: BTC/USDT
Current Price: 102624.40
1D Prediction: 103933.48
1W Prediction: 101866.88
RSI: 54.70
ATR: 4050.85
OBV: 139942.85


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - loss: 0.2335
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0260
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0179
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0133
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0099
Epoch 1/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.1689
Epoch 2/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0379
Epoch 3/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0241
Epoch 4/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0188
Epoch 5/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - loss: 0.0128
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step
Symbol: ETH/USDT
Current Price: 3142.23
1D Prediction: 3178.28
1W Prediction: 2952.97
RSI: 42.47
ATR: 179.60
OBV: 601376.20


Epoch 1/5


...

13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - loss: 0.0632
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0067
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0041
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0028
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0020
Epoch 1/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - loss: 0.0845
Epoch 2/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0326
Epoch 3/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0168
Epoch 4/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0214
Epoch 5/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0136
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 462ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 475ms/step
Symbol: XRP/USDT
Current Price: 3.12
1D Prediction: 3.03
1W Prediction: 1.30
RSI: 62.29
ATR: 0.22
OBV: 1770996559.14


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step - loss: 0.1187
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0196
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0112
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0080
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0056
Epoch 1/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 64ms/step - loss: 0.0305
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 0.0155
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.0148
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - loss: 0.0093
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.0082
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 474ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step
Symbol: LINK/USDT
Current Price: 23.40
1D Prediction: 24.14
1W Prediction: 19.17
RSI: 49.56
ATR: 2.09
OBV: 14893787.81


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.0560
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0147
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0084
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0067
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0059
Epoch 1/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - loss: 0.1761
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0451
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0521
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0215
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.0299
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 331ms/step
Symbol: INJ/USDT
Current Price: 18.48
1D Prediction: 20.38
1W Prediction: 18.62
RSI: 39.09
ATR: 2.10
OBV: 1543413.82


